# Use Sklearn models to train and predict RCC3 labels using Cross-validation

## Preamble

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold

from sklearn_models import run_train_sklearn_model



## Helper Functions and Classes

In [2]:
# Used to download data from dropbox and unzip it

def download_data_dir(dropbox_url, save_dir='data'):
    # Parse the file name from the URL
    file_name = dropbox_url.split("/")[-1]

    # Create the directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Send a GET request to the Dropbox URL
    response = requests.get(dropbox_url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        zip_path = os.path.join(save_dir, file_name)
        # Write the contents of the response to a file
        with open(zip_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)

        # Unzip the file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(save_dir)
    else:
        print(f"Failed to download data from {dropbox_url}. Status code: {response.status_code}")

    # delete the zip file
    os.remove(zip_path)
    return


# pytorch Dataset class for the data
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.to_numpy(), dtype=torch.float32)
        self.y = torch.tensor(y.to_numpy(), dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## Load (and explain) the Data

In [3]:
# subset_dir = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/March_8_Data'
subset_dir = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/March_6_Data'
# subset_dir = 'example_data'
example_data_url = 'https://www.dropbox.com/scl/fo/d1yqlmyafvu8tzujlg4nk/h?rlkey=zrxfapacmgb6yxsmonw4yt986&dl=1'
# Download the data

# download_data_dir(example_data_url, save_dir=subset_dir)

In [4]:
#what is inside the example directory?
print(os.listdir(subset_dir))

['Nivo Benefit BINARY finetune_folds', '.DS_Store', 'MSKCC BINARY finetune_folds', 'Cohort ID Expanded pretrain_folds', 'readme.txt', 'X.csv', 'y.csv', 'nans.csv']


#### X data
- the input peak matrix, columns are peaks, rows are samples
- nans are already imputed using the mean of the peak within its corresponding study
- in this example, the peaks are those determined by alignment
- also in this example, we are including a lot of pretraining data

In [5]:
X_data = pd.read_csv(os.path.join(subset_dir, 'X.csv'), index_col=0)

#### Nan mask
- nans is a binary matrix indicating which peaks were originally missing in which samples
- columns are the peaks, rows are the samples
- should have the exact same size and organization as X data
- Since nans were already imputed, this is not used in this example
- is useful if you want to change the imputation method, or calculate peak frequency

In [7]:
# nan_mask = pd.read_csv(os.path.join(subset_dir, 'nans.csv'), index_col=0)



#### Y data
- the metadata associated with the samples
- includes metadata from alignment with pre-training data AND metadata associated with the RCC studies
- RCC related metadata has been cleaned and simplified. combines the information between genomics dataset, the the Nature communication dataset. A lot of superfolous information has been removed
- the "Matt Set" column indicates whether a sample belongs in Matt's Train/Val/Split
- the "Set" column corresponds to the "Matt Set" for the RCC3 *baseline* samples, all other samples are labeled as pretrai
- columns have been created specifically for the Binary classification task "Nivo Benefit BINARY" and "MSKCC BINARY". 

In [6]:
y_data = pd.read_csv(os.path.join(subset_dir, 'y.csv'), index_col=0)

In [7]:
y_data['Matt Set'].value_counts()

Matt Set
Train    746
Other    425
Test     244
Val      229
Name: count, dtype: int64

In [7]:
y_data['Set'].value_counts()

Set
Pretrain      16944
Finetune        449
Test            149
Validation      143
Name: count, dtype: int64

In [8]:
print('number of samples: ', X_data.shape[0])
print('number of features: ', X_data.shape[1])

number of samples:  17685
number of features:  2736


In [9]:
pretrain_files = y_data[y_data['Set']=='Pretrain'].index.to_list()
finetune_files = y_data[y_data['Set']=='Finetune'].index.to_list()
holdout_test_files = y_data[y_data['Set']=='Test'].index.to_list()
holdout_val_files = y_data[y_data['Set']=='Validation'].index.to_list()

## Repeated Stratified Split of the Training data, using single column
this can be ignored since the data in the dropbox already has the splits defined
Creates a repeated stratified split of the training data to use cross-validation, saved in the "splits.csv" file

In [48]:
rskf_params = {
        'n_splits': 5,
        'n_repeats': 10,
        'random_state': 42,
    }


metadata_subset = y_data.loc[finetune_files].copy()
yes_remove_nans = True

stratify_col = 'Nivo Benefit BINARY'
# stratify_col = 'MSKCC BINARY'

splits_dir = os.path.join(subset_dir, f'{stratify_col} finetune_folds')
os.makedirs(splits_dir, exist_ok=True)

In [50]:
rskf = RepeatedStratifiedKFold(**rskf_params)

rskf_list = []

# check for nans
if metadata_subset[stratify_col].isna().any():
    if yes_remove_nans:
        metadata_subset = metadata_subset[~metadata_subset[stratify_col].isna()]
    else:
        print('There are nans in the stratify column')
        fill_val = metadata_subset[stratify_col].max() + 1
        metadata_subset[stratify_col] = metadata_subset[stratify_col].fillna(fill_val)




for i, (train_idx, test_idx) in enumerate(rskf.split(metadata_subset.index, metadata_subset[stratify_col])):

    test_bool = np.zeros(metadata_subset.shape[0], dtype=bool)
    test_bool[test_idx] = True
    rskf_list.append(test_bool)

rskf_df = pd.DataFrame(index=metadata_subset.index)
for i, test_idx in enumerate(rskf_list):
    rskf_df[f'fold_{i}'] = test_idx

rskf_df.to_csv(os.path.join(splits_dir, 'splits.csv'))

rskf_info = {
    'rskf_params': rskf_params,
    'stratify_col': stratify_col,
    'size': rskf_df.shape[0],
    'Train Bool' : False,
    'Test Bool' : True,
    'remove nans': yes_remove_nans
}

with open(os.path.join(splits_dir, 'splits_info.json'), 'w') as f:
    json.dump(rskf_info, f, indent=4)

## Run Classical Models on the desired tasks

In [10]:
# what are the available splits?

subset_dir_files = os.listdir(subset_dir)
split_dirs = [f for f in subset_dir_files if '_folds' in f]
print(split_dirs)


['Nivo Benefit BINARY finetune_folds', 'MSKCC BINARY finetune_folds', 'Cohort ID Expanded pretrain_folds']


### Create the task directory and properly format the X and y for the task

In [11]:
rcc3_baseline = y_data.loc[finetune_files].copy()

In [28]:
yes_dropna = True #drops nan values from the label column

# finetune_label_col = 'Benefit'
# finetune_label_mapper  = {'CB': 1, 'NCB': 0, 'ICB': np.nan}
# finetune_filter = rcc3_baseline[rcc3_baseline['Treatment'].isin(['NIVOLUMAB'])].index.to_list()


# finetune_label_col = 'MSKCC'
# finetune_label_mapper  = {'FAVORABLE': 1, 'POOR': 0, 'INTERMEDIATE': np.nan}
# finetune_filter = rcc3_baseline.index.to_list()

finetune_label_col = 'MSKCC BINARY'
finetune_label_mapper = {}
finetune_filter = []


# finetune_label_col = 'Nivo Benefit BINARY'
# finetune_label_mapper = {}
# finetune_filter = []

# desc_str = 'Nivo Benefit BINARY'
desc_str = 'MSKCC BINARY'
desc_str = finetune_label_col

splits_dir = os.path.join(subset_dir, f'{desc_str} finetune_folds')




print(finetune_label_col)
print(finetune_label_mapper)


task_dir = os.path.join(splits_dir, finetune_label_col)

MSKCC BINARY
{}


In [29]:

splits_df = pd.read_csv(os.path.join(splits_dir, 'splits.csv'), index_col=0)
if len(finetune_filter) > 0:
    print('number of samples after the finetune filter:' , len(finetune_filter))
    splits_df = splits_df.loc[finetune_filter].copy()

X = X_data.loc[splits_df.index]
y = y_data.loc[splits_df.index]
# nans = nan_mask.loc[splits_df.index].astype(bool)

n_folds = splits_df.shape[1]

task_info = {
    'label_col': finetune_label_col,
    'label_mapper': finetune_label_mapper,
    'filter': finetune_filter,
    'desc_str': desc_str,
    'size': splits_df.shape[0],
    'folds': splits_df.shape[1],
    'dropna': yes_dropna,
}

if finetune_label_mapper:
    task_info['size by label'] =  y[finetune_label_col].map(finetune_label_mapper).value_counts().to_dict(),
else:
    task_info['size by label'] =  y[finetune_label_col].value_counts().to_dict()

os.makedirs(task_dir, exist_ok=True)
with open(os.path.join(task_dir, 'task_info.json'), 'w') as f:
    json.dump(task_info, f, indent=4)

print(task_dir)

/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/March_6_Data/MSKCC BINARY finetune_folds/MSKCC BINARY


### cycle over the different models and run repeated cross-validation

NOTE: We are computed a weighted AUROC by default, weighted by class labels

In [30]:
val_hold_idx = holdout_val_files
Xval = X_data.loc[val_hold_idx]
yval = y_data.loc[val_hold_idx, finetune_label_col]
if yes_dropna:
    yval = yval.dropna()
    Xval = Xval.loc[yval.index]

In [31]:
output_dir = os.path.join(task_dir, 'classical_models')
if finetune_label_mapper:
    y_values = y[finetune_label_col].map(finetune_label_mapper)
else:
    y_values = y[finetune_label_col]
    
if yes_dropna:
    print('dropping nan values in the y column')
    y_values = y_values.dropna()
    X = X.loc[y_values.index]
    splits_df = splits_df.loc[y_values.index]

    print('number of samples after dropping nan values in the y column: ', y_values.shape[0])    


dropping nan values in the y column
number of samples after dropping nan values in the y column:  240


In [25]:
delete_other_files = False #delete the files generated from each fitting in the repeated Cross-validation
optimization_desc = 'default' #use sklearn's default hyperparameters
# optimization_desc = 'optimal' # use the default hyperparameter gridsearch specified in run_train_sklearn_model

which_models = ['logistic_regression', 'random_forest', 'svc']
# which_models = ['logistic_regression']


for model_kind in which_models:
    gather_output = []
    model_name = model_kind + '_' + optimization_desc
    output_summary_file = os.path.join(output_dir, f'{model_name}_summary.csv')
    
    if optimization_desc == 'default':
        param_grid = {}
    elif optimization_desc == 'optimal':
        param_grid = None # use the default hyperparameter gridsearch specified in run_train_sklearn_model

    for subset_id in range(n_folds):



        train_idx = splits_df.index[splits_df[f'fold_{subset_id}'] == False]
        test_idx = splits_df.index[splits_df[f'fold_{subset_id}'] == True]
        finetune_dataset = SimpleDataset(X.loc[train_idx], y_values.loc[train_idx])
        
        class_weights = 1 / torch.bincount(finetune_dataset.y.long())

        test_dataset = SimpleDataset(X.loc[test_idx], y_values.loc[test_idx])

        data_dict = {'train': finetune_dataset, 'CV': test_dataset}

        #if you want to add the validation set to the final evaluation
        data_dict['val'] = SimpleDataset(Xval, yval)

        ##### ##### ##### ##### ##### ##### ##### ##### ##### #####
        ##### Here is the code that actually runs the model optimization #####
        output_data = run_train_sklearn_model(data_dict,output_dir,
                                model_kind = model_kind,
                                model_name=f'{model_name}_{subset_id}',
                                param_grid=param_grid
                                )
        ##### ##### ##### ##### ##### ##### ##### ##### ##### #####
        

        gather_output.append(output_data)

    ## Summarize the important results
    # best_epoch_list = [output['best_epoch'] for output in gather_output]
        
    auc_summary_dct = {}
    for phase in data_dict.keys():
        auc_summary_dct[phase + ' AUROC'] = [output['end_state_auroc'][phase] for output in gather_output]

    # cv_auroc_list = [output['end_state_auroc']['CV'] for output in gather_output]
    # train_auroc_list = [output['end_state_auroc']['train'] for output in gather_output]
    # model_name = gather_output[0]['model_name']
    auc_summary = pd.DataFrame(auc_summary_dct)


    result_summary_avg = auc_summary.mean()
    result_summary_std = auc_summary.std()
    result_summary_avg.index = [f'AVG {col}' for col in result_summary_avg.index]
    result_summary_std.index = [f'STD {col}' for col in result_summary_std.index]
    result_summary = pd.concat([result_summary_avg, result_summary_std])
    result_summary.to_csv(output_summary_file)





output_files = os.listdir(output_dir)
output_summary_files = [f for f in output_files if f.endswith('summary.csv')]
other_files = [f for f in output_files if f not in output_summary_files]

all_res = []

for f in output_summary_files:
    print(f)
    model_name = f.split('_summary.csv')[0]
    res = pd.read_csv(os.path.join(output_dir, f), index_col=0)
    res.columns = [model_name]
    all_res.append(res)
    
# delete the other files to save room
if delete_other_files:
    for f in other_files:
        os.remove(os.path.join(output_dir, f))    

res_summary = pd.concat(all_res, axis=1)    
res_summary = res_summary.round(4)
res_summary.to_csv(os.path.join(task_dir, 'classical_summary.csv'))


dropping nan values in the y column
number of samples after dropping nan values in the y column:  156
svc_default_summary.csv
logistic_regression_default_summary.csv
random_forest_default_summary.csv


In [32]:
data_dict = {}
data_dict['train'] = SimpleDataset(X, y_values)
data_dict['test'] = SimpleDataset(Xval, yval)

for model_kind in which_models:

        model_name = model_kind + '_' + optimization_desc +' val'

        output_data = run_train_sklearn_model(data_dict,output_dir,
                                model_kind = model_kind,
                                model_name=f'{model_name}',
                                param_grid=param_grid
                                )
        
        valauc = output_data['end_state_auroc']['test']
        print(f'{model_name} val auc: {valauc}')
        

logistic_regression_default val val auc: 0.8607871532440186
random_forest_default val val auc: 0.931851327419281
svc_default val val auc: 0.9256560802459717


In [26]:
test_dataset

In [11]:
res_summary.round(3)

,svc_default,logistic_regression_default,logistic_regression_optimal,random_forest_default
AVG train AUROC,0.980,1.000,0.998,1.000
AVG CV AUROC,0.646,0.655,0.634,0.629
STD train AUROC,0.141,0.000,0.008,0.000
STD CV AUROC,0.090,0.074,0.080,0.074


In [21]:
auc_summary

,train AUROC,CV AUROC,val AUROC
0,0.999881,0.891841,0.908163
1,1.000000,0.857685,0.919096
2,1.000000,0.973435,0.924198
3,0.999761,0.884250,0.934402
4,0.999881,0.882812,0.930029
5,1.000000,0.941176,0.927114
6,1.000000,0.920304,0.910350
7,0.999761,0.861480,0.931487
8,1.000000,0.872865,0.917638
9,1.000000,0.845703,0.928572


In [14]:
# how to check if a variable is a function
callable(run_train_sklearn_model)

True

In [16]:
callable(subset_dir)

False

In [17]:
X_data.shape

(17685, 2736)

In [19]:
X_data.sum(axis=0)/X_data.shape[0]

FT10031   -0.002666
FT10035   -0.005164
FT10037   -0.003603
FT10039   -0.004585
FT10041   -0.007062
             ...   
FT9985    -0.007561
FT9988    -0.006673
FT9989    -0.004516
FT9997    -0.006612
FT9999    -0.008210
Length: 2736, dtype: float64